Training the decision making model

In [ ]:
import numpy as np
import pandas as pd
import copy
from sklearn.model_selection import train_test_split

In [ ]:
df=pd.read_csv('X_train_ordered.csv')
X=df.drop(columns=['Player Name','Team','pid','role','bid team','output','fair'])
X.head()

,Price,isoversea,isCap,isbatsman,isbowler,iswicketkeeper,base,runs_per_match,matches_bat,strike_rate_bat,...,bat_death,bowl_powerplay,bowl_death,supply,no overseas,demand_team,demand_other,purse bid,max purse,avg purse
0,8.25,0,1,1,0,0,2.0,31.086957,92,134.335369,...,138.906803,0.0,0.0,1,1.0,4.0,29.0,48.0,72.0,55.2875
1,8.25,0,1,1,0,0,2.0,31.086957,92,134.335369,...,138.906803,0.0,0.0,1,1.0,4.0,29.0,47.5,72.0,55.3500
2,8.25,0,1,1,0,0,2.0,31.086957,92,134.335369,...,138.906803,0.0,0.0,1,1.0,2.0,31.0,52.0,72.0,54.7875
3,8.25,0,1,1,0,0,2.0,31.086957,92,134.335369,...,138.906803,0.0,0.0,1,2.0,6.0,27.0,48.0,72.0,55.2875
4,8.25,0,1,1,0,0,2.0,31.086957,92,134.335369,...,138.906803,0.0,0.0,1,1.0,3.0,30.0,59.8,72.0,53.8125


Adding additional data using the fact that a team that bids for a player at price p would also bid for the player at price less than p.

In [ ]:
df=pd.read_csv('X_train_ordered.csv')
new_df=pd.DataFrame()
for i in range(len(df)):
  current=df.iloc[i]
  if current['Team']=='Unsold':
    current['Price']=current['base']
    new_df=new_df.append(current,ignore_index=True)

  else:
    if current['output']==1:
      for m in range(1,20):
        temp=copy.deepcopy(current)
        temp['Price']=(current['Price']*m)/19.0
        new_df=new_df.append(temp,ignore_index=True)
    else:

      new_df=new_df.append(current,ignore_index=True)


<ipython-input-3-08cfa15e5ece>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df=new_df.append(current,ignore_index=True)
<ipython-input-3-08cfa15e5ece>:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df=new_df.append(temp,ignore_index=True)
<ipython-input-3-08cfa15e5ece>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  current['Price']=current['base']
<ipython-input-3-08cfa15e5ece>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_df=new_df.append(current,ignore_index=True)


In [ ]:
new_df.to_csv('output.csv',index=False)

In [ ]:
X=new_df.drop(columns=['Player Name','Team','pid','role','bid team','output','fair'])
y=new_df['output']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #train val split

In [ ]:
#Training various ML classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


models = [
    LogisticRegression(),
    RandomForestClassifier(),
    SVC(probability=True),
    XGBClassifier(n_estimators=1000, max_depth=7, learning_rate=0.1, subsample=0.7, colsample_bytree=0.8)
]

result = pd.DataFrame(columns=['name', 'Train Accuracy', 'Test Accuracy'])

for model in models:
    model.fit(X_train, y_train)
    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)

    result = result.append({'name': str(model), 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)

print(result)
#Not saving logistic and SVM since the results are poor, other models are saved through the below code block

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
<ipython-input-7-a8af98b5c07c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': str(model), 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)
<ipython-input-7-a8af98b5c07c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.app

                                                name  Train Accuracy  \
0                               LogisticRegression()        0.702598   
1                           RandomForestClassifier()        1.000000   
2                              SVC(probability=True)        0.589487   
3  XGBClassifier(base_score=None, booster=None, c...        1.000000   

   Test Accuracy  
0       0.701554  
1       0.979558  
2       0.605070  
3       0.968929  


<ipython-input-7-a8af98b5c07c>:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': str(model), 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


Training some more classifiers

In [ ]:
!pip install lightgbm catboost
from sklearn.ensemble import VotingClassifier, AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier, HistGradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import joblib

# Assuming X_train, X_test, y_train, and y_test are your training and testing data
# Replace them with your actual data

# Instantiate individual classifiers with default values
adaboost = AdaBoostClassifier(random_state=42)
gradboost = GradientBoostingClassifier(random_state=42)
xgb = XGBClassifier(random_state=42)
lgbm = LGBMClassifier(random_state=42)
catboost = CatBoostClassifier(random_state=42, verbose=0)
hist_gradient_boost = HistGradientBoostingClassifier(random_state=42)
random_forest = RandomForestClassifier(random_state=42)

# Create an ensemble of classifiers
models = [
    ('AdaBoost', adaboost),
    ('GradientBoost', gradboost),
    ('XGBoost', xgb),
    ('LightGBM', lgbm),
    ('CatBoost', catboost),
    ('HistGradientBoost', hist_gradient_boost),
    ('RandomForest', random_forest)
]

# Train and save each individual classifier
result = pd.DataFrame(columns=['name', 'Train Accuracy', 'Test Accuracy'])

for name, model in models:
    model.fit(X_train, y_train)

    # Save the trained model to a file
    filename = f"{name}_model.sav"
    joblib.dump(model, filename)

    train_predictions = model.predict(X_train)
    test_predictions = model.predict(X_test)

    train_accuracy = accuracy_score(y_train, train_predictions)
    test_accuracy = accuracy_score(y_test, test_predictions)

    result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)

# Create an ensemble of classifiers
ensemble_classifier = VotingClassifier(estimators=models, voting='soft')

# Train the ensemble classifier
ensemble_classifier.fit(X_train, y_train)

# Save the ensemble model to a file
ensemble_filename = "ensemble_model.sav"
joblib.dump(ensemble_classifier, ensemble_filename)

# Predictions on the training and testing data
train_predictions = ensemble_classifier.predict(X_train)
test_predictions = ensemble_classifier.predict(X_test)

# Evaluate accuracy for the ensemble
train_accuracy = accuracy_score(y_train, train_predictions)
test_accuracy = accuracy_score(y_test, test_predictions)

# Display the results
result = result.append({'name': 'Ensemble', 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)

print(result)



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 7.5 MB/s eta 0:00:00


<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)
<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)
<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2861, number of negative: 2028
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000875 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3031
[LightGBM] [Info] Number of data points in the train set: 4889, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.585191 -> initscore=0.344121
[LightGBM] [Info] Start training from score 0.344121


<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)
<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)
<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': name, 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)
<ipython-input-8-b99419e78912>:50: FutureWarning: The frame.append method is deprecated and will be removed from panda

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 2861, number of negative: 2028
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002713 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3031
[LightGBM] [Info] Number of data points in the train set: 4889, number of used features: 27
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.585191 -> initscore=0.344121
[LightGBM] [Info] Start training from score 0.344121
                name  Train Accuracy  Test Accuracy
0           AdaBoost        0.856617       0.847097
1      GradientBoost        0.922070       0.907604
2            XGBoost        0.998773       0.966476
3           LightGBM        0.991409       0.968111
4           CatBoost        0.986909       0.966476
5  HistGradientBoost        0.991000       0.970564
6       RandomForest        1.000000       0.976288
7           Ens

<ipython-input-8-b99419e78912>:71: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result = result.append({'name': 'Ensemble', 'Train Accuracy': train_accuracy, 'Test Accuracy': test_accuracy}, ignore_index=True)


In [ ]:
import sklearn
sklearn.metrics.confusion_matrix(y_test,test_predictions,normalize='true')

array([[0.96579477, 0.03420523],
       [0.06609195, 0.93390805]])